In [1]:
from toolkits.datasets import ShortTermTrafficDataset, load_next_30min
from toolkits.datapreparing import genSamples, groupVDs
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
from datetime import datetime
import numpy as np
import pandas as pd
import feather
import torch
import torch.nn as nn
import os

C:\Users\Home\AppData\Local\Temp\ipykernel_50104\980680391.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
class CNNModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cnnLayer = nn.Sequential(
            nn.Conv2d(in_channels=5, out_channels=16, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),
        )

        self.fcLayer = nn.Sequential(
            nn.Linear(32 * 1 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 12),
            nn.Sigmoid(),
        )

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(self.device)

    def forward(self, x) -> torch.Tensor:
        x = self.cnnLayer(x)
        x = x.flatten(1)
        x = self.fcLayer(x)
        return x

    def save(self, ckpt_dir: str) -> None:
        torch.save(self.state_dict(), os.path.join(ckpt_dir, 'model.pth'))

    def load(self, ckpt_dir: str) -> None:
        self.load_state_dict(torch.load(os.path.join(ckpt_dir, 'model.pth')))

In [3]:
totalDateRange = list(map(lambda x: str(x)[:10], pd.date_range('2024-01-01', '2024-02-25')))
lunarNewYear = list(map(lambda x: str(x)[:10], pd.date_range('2024-02-08', '2024-02-14')))
natlHoliday = ['2024-01-01']
workSat = '2024-02-17'

remainDate = list(set(totalDateRange) - set(lunarNewYear) - set(natlHoliday))
weekends = [x for x in remainDate if datetime.strptime(x, '%Y-%m-%d').isoweekday() > 5 and x != workSat]
weekdays = [x for x in remainDate if datetime.strptime(x, '%Y-%m-%d').isoweekday() <= 5] + [workSat]

weekends = pd.Series(weekends).sort_values().to_list()
weekdays = pd.Series(weekdays).sort_values().to_list()

In [6]:
dateTypes = ['weekdays', 'weekends', 'natlHoliday', 'lunarNewYear']
directions = ['S', 'N']
roadSection = [{'Start': '南港系統', 'End': '石碇'},
               {'Start': '石碇', 'End': '坪林'},
               {'Start': '坪林', 'End': '頭城'},
               {'Start': '頭城', 'End': '宜蘭'},
               {'Start': '宜蘭', 'End': '羅東'},
               {'Start': '羅東', 'End': '蘇澳'}]

In [7]:
for datetype in dateTypes:
    df = feather.read_dataframe(f"./nfb2024/{datetype}/{datetype}.feather")
    for direct in directions:
        for section in roadSection:
            filtDf = df.loc[
                (df['RoadDirection']==direct) &\
                (df['Start'].str.contains(section['Start'])) &\
                (df['End'].str.contains(section['End']))
            ].reset_index(drop=True)

            speedCollection, volCollection, occCollection, laneCollection, tunnelCollection =\
                [], [], [], [], []
            vdGrps = groupVDs(filtDf, each=3)
            for groupKey in vdGrps.keys():
                speeds, vols, occs, lanes, tunnels = genSamples(filtDf, vdGrps, groupKey,
                                                                each=3, preDuration=30, postDuration=30)
                speedCollection += speeds
                volCollection += vols
                occCollection += occs
                laneCollection += lanes
                tunnelCollection += tunnels

            # Create Dataset
            validDataset = ShortTermTrafficDataset(speed_data=speedCollection, volume_data=volCollection, occupy_data=occCollection,
                                       lane_data=laneCollection, tunnel_data=tunnelCollection, load_ckpt=False, mode='valid',
                                       ckpt_dir='../hwyTrafficPred/toolkits/next_half_hour_data')
            
            valDataLoader = DataLoader(validDataset, batch_size=1024, shuffle=False)
            model = CNNModel()
            model.load('./model/')
            model.eval()

            speed_records = []
            volume_records = []

            for batch in tqdm(valDataLoader):
                X, y = batch
                with torch.no_grad():
                    logits = model(X.to(model.device))
                label_speeds = np.array(y)[:,0] * 100
                label_volumes = np.array(y)[:,1] * 250
                pred_speeds = np.array(logits)[:,0] * 100
                pred_volumes = np.array(logits)[:,1] * 250

                # filt valid indices
                indices_speeds = np.where(label_speeds!=0)[0]
                indices_volumes = np.where(label_volumes!=0)[0]

                label_speeds = label_speeds[indices_speeds]
                label_volumes = label_volumes[indices_volumes]
                pred_speeds = pred_speeds[indices_speeds]
                pred_volumes = pred_volumes[indices_volumes]

                speed_records.append((np.abs(pred_speeds - label_speeds) / label_speeds).mean())
                volume_records.append((np.abs(pred_volumes - label_volumes) / label_volumes).mean())

            print('==============================================================', end='\n')
            print(f"DateType: {datetype} | Direction: {direct} | Section: {section}", end='\n')
            print(f"MAPE (speed): {np.array(speed_records).mean():.2%}", end='\n')
            print(f"MAPE (volume): {np.array(volume_records).mean():.2%}", end='\n')
            print('==============================================================', end='\n\n')

100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


DateType: natlHoliday | Direction: S | Section: {'Start': '南港系統', 'End': '石碇'}
MAPE (speed): 1.91%
MAPE (volume): 1.95%



100%|██████████| 3/3 [00:02<00:00,  1.42it/s]


DateType: natlHoliday | Direction: S | Section: {'Start': '石碇', 'End': '坪林'}
MAPE (speed): 3.51%
MAPE (volume): 3.50%



100%|██████████| 5/5 [00:03<00:00,  1.46it/s]


...

DateType: lunarNewYear | Direction: N | Section: {'Start': '頭城', 'End': '宜蘭'}
MAPE (speed): 12.85%
MAPE (volume): 18.24%



100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


DateType: lunarNewYear | Direction: N | Section: {'Start': '宜蘭', 'End': '羅東'}
MAPE (speed): 3.92%
MAPE (volume): 3.95%



100%|██████████| 4/4 [00:02<00:00,  1.47it/s]

DateType: lunarNewYear | Direction: N | Section: {'Start': '羅東', 'End': '蘇澳'}
MAPE (speed): 10.25%
MAPE (volume): 10.42%

